In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np

features_for_mood = ['energy', 'liveness', 'tempo', 'speechiness',
                                     'acousticness', 'instrumentalness', 'danceability', 'duration_ms',
                                     'loudness', 'valence']

data = pd.read_csv('/content/drive/MyDrive/music-mood-classification-main/tracks.csv')

In [4]:
trainx, testx, trainy, testy = train_test_split(data[features_for_mood], data['mood'], test_size = 0.33,
                                                random_state = 42, stratify=data['mood'])

In [11]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(trainx[features_for_mood])
test_scaled = scaler.fit_transform(testx[features_for_mood])
logreg = LogisticRegression(max_iter=2000)
logreg.fit(train_scaled, trainy)
print (accuracy_score(logreg.predict(train_scaled), trainy))

0.7027863777089783


In [12]:
scores = cross_val_score(logreg, train_scaled, trainy, cv=5)
print (scores.mean())

0.6532692307692308


In [13]:
params = {"C" : np.logspace(-6, 3, 10)}
clf = GridSearchCV(logreg, params)
clf.fit(train_scaled, trainy)
print (clf.best_estimator_.C)
print (clf.best_score_)

0.1
0.6625480769230769


In [14]:
logreg = LogisticRegression(max_iter=2000, C=0.1)
logreg.fit(train_scaled, trainy)
preds = clf.predict(test_scaled)
print (accuracy_score(preds, testy))

0.6


In [15]:
import math

In [16]:
fi = pd.DataFrame(clf.best_estimator_.coef_, columns=features_for_mood)
pow(math.e, fi)
fo = fi.set_axis(logreg.classes_, axis=0)
fo
fo.idxmax(axis=1)

Aggressive         speechiness
Dark              acousticness
Energetic               energy
Happy                  valence
Relaxing      instrumentalness
dtype: object

In [17]:
fo

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,danceability,duration_ms,loudness,valence
Aggressive,0.567769,0.064396,0.088847,0.580129,-0.195869,-0.139348,0.283104,0.093854,0.252611,-0.435541
Dark,-0.369161,-0.010836,-0.053483,-0.119948,0.299276,-0.663417,-0.329764,0.287847,0.184686,-0.378019
Energetic,0.418039,0.099118,0.049743,-0.139246,-0.527507,0.352835,-0.254538,0.299224,0.093253,0.379563
Happy,-0.199490,-0.057571,0.124830,-0.238279,0.056677,-0.227825,0.675224,-0.119654,0.037214,0.724735
Relaxing,-0.417157,-0.095106,-0.209936,-0.082657,0.367423,0.677756,-0.374025,-0.561270,-0.567763,-0.290738


In [19]:
fo.to_csv('/content/drive/MyDrive/music-mood-classification-main/fo.csv', sep='\t')